In [12]:
import os
import datetime
import pprint
import hashlib
import xlrd
import dataset

In [13]:
def dated_file_name(extension, prefix):
    filename = prefix + "_" + datetime.datetime.now().isoformat() + "." + extension
    return filename

In [14]:
def hash(path, buffer_size=64*2**10):
    sha1 = hashlib.sha1()
    with open(path, "rb") as f:
        buffer = f.read(buffer_size)
        while len(buffer) > 0:
            sha1.update(buffer)
            buffer = f.read(buffer_size)
    return sha1.hexdigest()

In [15]:
class KnownFile(object):
    def __init__(self, path, persister=None):
        self.path = path
        self.exists = os.path.exists(path)
        if self.exists:
            self.realpath = os.path.realpath(path)
            self.isfile = os.path.isfile(self.realpath)
            if self.isfile:
                self.sha1 = hash(self.realpath)
                if persister is not None:
                    self.persister = persister
                    persister.persist(self)
                    
    def __repr__(self):
        return str((self.__class__.__name__, vars(self)))

In [21]:
def kf(path):
    return KnownFile(path).realpath

In [16]:
class MappingFile(object):
    def __repr__(self):
        return str((self.__class__.__name__, vars(self)))

In [31]:
data_dir = os.path.expanduser("~/dev/mkgu_packaging/data")
data_dir

'/braintree/home/jjpr/dev/mkgu_packaging/data'

In [19]:
# mf_paths = [os.path.join(data_dir, x) for x in os.listdir(data_dir) if x.endswith(".xls")]
mf_paths = [
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-000936/5602-2 SN1025-000936.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0547,548,549/090609B-5E.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0547,548,549/090609B-5F.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0547,548,549/090609B-5H.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0658, 0656, 0657/2132-5-SN0656.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0658, 0656, 0657/2132-8-SN0657.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0658, 0656, 0657/2466-1-SN0658.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-552,559,649/090609B-4F.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-552,559,649/090609B-8E.xls",
    "/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-552,559,649/2467-6_SN0649.xls"    
]
mf_knownfiles = [KnownFile(x) for x in mf_paths]
mf_knownfiles

[('KnownFile', {'path': '/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-000936/5602-2 SN1025-000936.xls', 'exists': True, 'realpath': '/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-000936/5602-2 SN1025-000936.xls', 'isfile': True, 'sha1': '6f31b351f851c75364be07386c3120ea87b0b2d2'}),
 ('KnownFile', {'path': '/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0547,548,549/090609B-5E.xls', 'exists': True, 'realpath': '/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0547,548,549/090609B-5E.xls', 'isfile': True, 'sha1': 'fc0d9e2aa9887d06adc52b2dd6760b93bff6fc38'}),


In [4]:
def is_comment_row(row):
    return str(row[0].value).lstrip().startswith("//")

In [5]:
def row_col_valid(cell):
    if cell.ctype != 2: return False
    if type(cell.value) != float: return False
    if cell.value % 1 != 0: return False
    if cell.value < 0: return False
    if cell.value > 9: return False
    return True

In [52]:
def get_mapping(i, row):
    if not (row_col_valid(row[0]) and row_col_valid(row[1])):
        return None
    result = {
        "electrode_column": int(row[0].value), 
        "electrode_row": int(row[1].value), 
        "bank_name": row[2].value,
        "electrode_number_in_bank": int(row[3].value),
        "electrode_label": row[4].value,
        "electrode_label_numeric": int(row[4].value[4:]),
        "index_in_mapping_sheet": i
    }
    return result

In [24]:
def jump_over(input_str, to_jump):
    if input_str.startswith(to_jump):
        return input_str[len(to_jump):]
    else:
        raise ValueError("'{0}' does not start with '{1}'".format(input_str, to_jump))

In [25]:
def get_array_id(row):
    stripped = row[0].value.strip()
    chopped = jump_over(stripped, "Cerebus mapping for array ")
    squoze = ''.join(chopped.split())
    return squoze

In [8]:
def get_mapping_file(path):
    mf = MappingFile()
    mf.xls_path = path
    mf.workbook = xlrd.open_workbook(path)
    mf.mapping_worksheet = mf.workbook.sheet_by_name("Cerebus mapping")
    mf.comment_lines = set()
    mf.description_row = None
    mf.mappings = []
    for i, row in enumerate(mf.mapping_worksheet.get_rows()):
        if is_comment_row(row):
            mf.comment_lines.add(i)
        elif mf.description_row is None:
            mf.description_row = row
        else:
            mapping = get_mapping(i, row)
            if mapping is not None:
                mf.mappings.append(mapping)
    mf.array_id = get_array_id(mf.description_row)
    return mf

In [53]:
mfs = []

for path in mf_paths:
    mf = get_mapping_file(kf(path))
    mfs.append(mf)

pprint.pprint([vars(mf) for mf in mfs])
# mfs

[{'array_id': '5602-2/SN1025-000936',
  'comment_lines': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11},
  'description_row': [text:'Cerebus mapping for array 5602-2/SN1025-000936',
                      empty:'',
                      empty:'',
                      empty:'',
                      empty:'',
                      empty:'',
                      text:'elec',
                      empty:''],
  'mapping_worksheet': <xlrd.sheet.Sheet object at 0x7fe5dc0e6b38>,
  'mappings': [{'bank_name': 'A',
                'electrode_column': 8,
                'electrode_label': 'elec99',
                'electrode_label_numeric': 99,
                'electrode_number_in_bank': 14,
                'electrode_row': 9,
                'index_in_mapping_sheet': 13},
               {'bank_name': 'A',
                'electrode_column': 7,
                'electrode_label': 'elec98',
                'electrode_label_numeric': 98,
                'electrode_number_in_bank': 10,
                'elec

 {'array_id': '090609B-5F/SN0548',
  'comment_lines': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11},
  'description_row': [text:'Cerebus mapping for array 090609B-5F/SN0548',
                      empty:'',
                      empty:'',
                      empty:'',
                      empty:'',
                      empty:'',
                      text:'elec',
                      empty:''],
  'mapping_worksheet': <xlrd.sheet.Sheet object at 0x7fe5ca0ed630>,
  'mappings': [{'bank_name': 'A',
                'electrode_column': 8,
                'electrode_label': 'elec99',
                'electrode_label_numeric': 99,
                'electrode_number_in_bank': 14,
                'electrode_row': 9,
                'index_in_mapping_sheet': 13},
               {'bank_name': 'A',
                'electrode_column': 7,
                'electrode_label': 'elec98',
                'electrode_label_numeric': 98,
                'electrode_number_in_bank': 10,
                'electrode_

                'electrode_number_in_bank': 22,
                'electrode_row': 7,
                'index_in_mapping_sheet': 32},
               {'bank_name': 'A',
                'electrode_column': 8,
                'electrode_label': 'elec79',
                'electrode_label_numeric': 79,
                'electrode_number_in_bank': 20,
                'electrode_row': 7,
                'index_in_mapping_sheet': 33},
               {'bank_name': 'A',
                'electrode_column': 7,
                'electrode_label': 'elec78',
                'electrode_label_numeric': 78,
                'electrode_number_in_bank': 23,
                'electrode_row': 7,
                'index_in_mapping_sheet': 34},
               {'bank_name': 'A',
                'electrode_column': 6,
                'electrode_label': 'elec77',
                'electrode_label_numeric': 77,
                'electrode_number_in_bank': 13,
                'electrode_row': 7,
                'index_in_ma

                'electrode_label_numeric': 9,
                'electrode_number_in_bank': 32,
                'electrode_row': 0,
                'index_in_mapping_sheet': 101},
               {'bank_name': 'C',
                'electrode_column': 7,
                'electrode_label': 'elec8',
                'electrode_label_numeric': 8,
                'electrode_number_in_bank': 29,
                'electrode_row': 0,
                'index_in_mapping_sheet': 102},
               {'bank_name': 'C',
                'electrode_column': 6,
                'electrode_label': 'elec7',
                'electrode_label_numeric': 7,
                'electrode_number_in_bank': 28,
                'electrode_row': 0,
                'index_in_mapping_sheet': 103},
               {'bank_name': 'C',
                'electrode_column': 5,
                'electrode_label': 'elec6',
                'electrode_label_numeric': 6,
                'electrode_number_in_bank': 26,
                'elec

               {'bank_name': 'C',
                'electrode_column': 5,
                'electrode_label': 'elec6',
                'electrode_label_numeric': 6,
                'electrode_number_in_bank': 26,
                'electrode_row': 0,
                'index_in_mapping_sheet': 103},
               {'bank_name': 'C',
                'electrode_column': 4,
                'electrode_label': 'elec5',
                'electrode_label_numeric': 5,
                'electrode_number_in_bank': 24,
                'electrode_row': 0,
                'index_in_mapping_sheet': 104},
               {'bank_name': 'C',
                'electrode_column': 3,
                'electrode_label': 'elec4',
                'electrode_label_numeric': 4,
                'electrode_number_in_bank': 21,
                'electrode_row': 0,
                'index_in_mapping_sheet': 105},
               {'bank_name': 'C',
                'electrode_column': 2,
                'electrode_label': 'elec3'

                'electrode_label_numeric': 34,
                'electrode_number_in_bank': 15,
                'electrode_row': 3,
                'index_in_mapping_sheet': 77},
               {'bank_name': 'B',
                'electrode_column': 2,
                'electrode_label': 'elec33',
                'electrode_label_numeric': 33,
                'electrode_number_in_bank': 13,
                'electrode_row': 3,
                'index_in_mapping_sheet': 78},
               {'bank_name': 'C',
                'electrode_column': 1,
                'electrode_label': 'elec32',
                'electrode_label_numeric': 32,
                'electrode_number_in_bank': 12,
                'electrode_row': 3,
                'index_in_mapping_sheet': 79},
               {'bank_name': 'C',
                'electrode_column': 0,
                'electrode_label': 'elec31',
                'electrode_label_numeric': 31,
                'electrode_number_in_bank': 11,
                '

In [27]:
[x.array_id for x in mfs]

['5602-2/SN1025-000936',
 '090609B-5E/SN00547',
 '090609B-5F/SN0548',
 '090609B-5H/SN0549',
 '2132-5/SN0656',
 '2132-8/SN0657',
 '2466-1/SN0658',
 '090609B-4F/SN0559',
 '090609B-8E/SN0552',
 '2467-6/SN0649']

In [35]:
try: 
    db
except NameError:
    db_file = os.path.join(data_dir, dated_file_name("db", "array_meta"))
    db_url = "sqlite:///" + db_file
    db = dataset.connect(db_url)

In [36]:
# del db
db

<Database(sqlite:////braintree/home/jjpr/dev/mkgu_packaging/data/array_meta_2018-02-13T15:33:27.766141.db)>

In [59]:
tbl_array = db["array"]
tbl_mapping = db["mapping"]
for mf in mfs:
    array_dict = {"blackrock_id_from_xls": mf.array_id, "mapping_xls":mf.xls_path}
    array_id = tbl_array.upsert(array_dict, keys=["blackrock_id_from_xls"])
    if type(array_id) == bool:
        array_id = tbl_array.find_one(blackrock_id_from_xls=mf.array_id)["id"]
    print(array_id)
    for mapping in mf.mappings:
        insert = dict(mapping)
        insert["array_id"] = array_id
        tbl_mapping.upsert(insert, keys=["array_id", "electrode_column", "electrode_row"])

1
2
3
4
5
6
7
8
9
10


In [58]:
# tbl_mapping.drop()

In [39]:
animals = [
    {
        "name": "Tito",
        "implants": [
            {"hemisphere": "L", "position": "A", "blackrock_id_from_surgery_note": "2132-8/SN0657",
            "info_source": "/braintree/data1/archive/animals/Tito/mindhive/dicarlolab/data/Animals/07-01 Tito/20110628_implant_left_hemisphere/surgery_note/surgery_note.pages"},
            {"hemisphere": "L", "position": "M", "blackrock_id_from_surgery_note": "2467-6/SN0649",
            "info_source": "/braintree/data1/archive/animals/Tito/mindhive/dicarlolab/data/Animals/07-01 Tito/20110628_implant_left_hemisphere/surgery_note/surgery_note.pages"},
            {"hemisphere": "L", "position": "P", "blackrock_id_from_surgery_note": "2132-5/SN0656",
            "info_source": "/braintree/data1/archive/animals/Tito/mindhive/dicarlolab/data/Animals/07-01 Tito/20110628_implant_left_hemisphere/surgery_note/surgery_note.pages"},
        ]
    },
    {
        "name": "Chabo",
        "implants": [
            {"hemisphere": "L", "position": "A", "blackrock_id_from_surgery_note": "SN1024-0547",
            "info_source": "/braintree/data1/archive/animals/Chabo/mindhive/dicarlolab/data/Animals/07-02 Chabo/CHABO_010511_surgery_left/surgery_note/surgery_note.pages"},
            {"hemisphere": "L", "position": "M", "blackrock_id_from_surgery_note": "SN1025-0549",
            "info_source": "/braintree/data1/archive/animals/Chabo/mindhive/dicarlolab/data/Animals/07-02 Chabo/CHABO_010511_surgery_left/surgery_note/surgery_note.pages"},
            {"hemisphere": "L", "position": "P", "blackrock_id_from_surgery_note": "SN1025-0552",
            "info_source": "/braintree/data1/archive/animals/Chabo/mindhive/dicarlolab/data/Animals/07-02 Chabo/CHABO_010511_surgery_left/surgery_note/surgery_note.pages"},
        ]
    }
]

In [40]:
# I am just guessing.
blackrock_id_map = [
    {"blackrock_id_from_surgery_note": "2132-8/SN0657", "blackrock_id_from_xls": "2132-8/SN0657"},
    {"blackrock_id_from_surgery_note": "2467-6/SN0649", "blackrock_id_from_xls": "2467-6/SN0649"},
    {"blackrock_id_from_surgery_note": "2132-5/SN0656", "blackrock_id_from_xls": "2132-5/SN0656"},
    {"blackrock_id_from_surgery_note": "SN1024-0547", "blackrock_id_from_xls": "090609B-5E/SN00547"},
    {"blackrock_id_from_surgery_note": "SN1025-0549", "blackrock_id_from_xls": "090609B-5H/SN0549"},
    {"blackrock_id_from_surgery_note": "SN1025-0552", "blackrock_id_from_xls": "090609B-8E/SN0552"}
]

In [41]:
tbl_animal = db["animal"]
tbl_implant = db["implant"]
for animal in animals:
    animal_id = tbl_animal.upsert({"name": animal["name"]}, keys=["name"])
    if type(animal_id) == bool:
        animal_id = tbl_animal.find_one(name=animal["name"])["id"]
    print(animal_id)
    for implant in animal["implants"]:
        id_map = next(x for x in blackrock_id_map if x["blackrock_id_from_surgery_note"]==implant["blackrock_id_from_surgery_note"])
        array_id = tbl_array.find_one(blackrock_id_from_xls=id_map["blackrock_id_from_xls"])["id"]
        insert = {
            "animal_id": animal_id, 
            "array_id": array_id, 
            "hemisphere": implant["hemisphere"], 
            "position": implant["position"],
            "info_source": implant["info_source"],
            "blackrock_id_from_surgery_note": implant["blackrock_id_from_surgery_note"]
        }
        tbl_implant.upsert(insert, keys=["animal_id", "array_id"])

1
2


In [60]:
sql_overview = (
    "SELECT "
    "* "
    "FROM "
    "animal an "
    "JOIN implant i ON i.animal_id = an.id "
    "JOIN array ar ON i.array_id = ar.id "
    "JOIN mapping m ON m.array_id = ar.id"
    ";"
)
big_overview = list(db.query(sql_overview))
big_overview

[OrderedDict([('id', 576),
              ('name', 'Tito'),
              ('animal_id', 1),
              ('array_id', 6),
              ('hemisphere', 'L'),
              ('position', 'A'),
              ('info_source',
               '/braintree/data1/archive/animals/Tito/mindhive/dicarlolab/data/Animals/07-01 Tito/20110628_implant_left_hemisphere/surgery_note/surgery_note.pages'),
              ('blackrock_id_from_surgery_note', '2132-8/SN0657'),
              ('blackrock_id_from_xls', '2132-8/SN0657'),
              ('mapping_xls',
               '/braintree/data2/active/users/darren/mindhive/dicarlolab/u/darren/mworks_array_xmls/preproc/orig_hvm/array_maps_from_blackrock/Mapping and Impedance - 1025-0658, 0656, 0657/2132-8-SN0657.xls'),
              ('electrode_column', 0),
              ('electrode_row', 0),
              ('bank_name', 'C'),
              ('electrode_number_in_bank', 17),
              ('electrode_label', 'elec1'),
              ('electrode_label_numeric', 1),
  

In [62]:
# should be 6 implanted arrays * 96 electrodes = 576
len(big_overview)

576